In [2]:
# Base Imports

import operator
import random

from deap import base, gp, tools

# Requires installation of Graphviz-dev tool
import pygraphviz as pgv



In [3]:
# Defining a primitive set
pset = gp.PrimitiveSet("main", 2) # name of the procedure generated, and its inputs

# Adding Operators to the Primitive Set
pset.addPrimitive(max, 2)      # add the max function as a primitive with 2 inputs
pset.addPrimitive(operator.add, 2) # add the addition operator (2 inputs)
pset.addPrimitive(operator.mul, 2) # add the mult operator (2 inputs)
pset.addPrimitive(operator.neg, 1) # add the negation operator (1 input)

# Adding Terminals to the Primitive Set
pset.renameArguments(ARG0="x") # Setting the input var name
pset.renameArguments(ARG1="y") # Setting the input var name
pset.addTerminal(1)            # Fixed value terminal

In [4]:
# Generating a Tree
expr = gp.genFull(pset, min_=1, max_=3) # Generates a rendom full tree from a terminal set
tree = gp.PrimitiveTree(expr)           # Tree constructor

print(tree)

max(neg(max(y, 1)), max(mul(x, y), mul(x, x)))


In [5]:
# Tree Visualization
nodes, edges, labels = gp.graph(tree)

print(nodes)
print(edges)
print(labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[(0, 1), (1, 2), (2, 3), (2, 4), (0, 5), (5, 6), (6, 7), (6, 8), (5, 9), (9, 10), (9, 11)]
{0: 'max', 1: 'neg', 2: 'max', 3: 'y', 4: 1, 5: 'max', 6: 'mul', 7: 'x', 8: 'y', 9: 'mul', 10: 'x', 11: 'x'}


In [6]:
# Graphviz

def drawtree(t, filename):
    nodes, edges, labels = gp.graph(t)
    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(filename)

drawtree(tree, "tree.png")


In [7]:
# Running a tree
tree_exec = gp.compile(tree, pset)

# Checking results:
print(tree)
print(tree_exec(1, 2)) # Remember we defined two inputs

max(neg(max(y, 1)), max(mul(x, y), mul(x, x)))
2


In [8]:
# We can create trees directly from strings.

tree2 = gp.PrimitiveTree.from_string("mul(x, y)", pset)
print(tree2)

tree2_exec = gp.compile(tree2, pset)
print(tree2_exec(3, 2))

drawtree(tree2, "tree2.png")

mul(x, y)
6
